In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from utils import find_length, find_length_diff
from TSB_AD.utils.slidingWindows import find_length_rank
from scipy import stats
import pandas as pd
from statsmodels.tsa.seasonal import STL
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import acf
from scipy.signal import find_peaks, peak_prominences, peak_widths
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
np.set_printoptions(legacy='1.13')

In [ ]:
def random_walk(length=10000):
    return np.cumsum(np.random.uniform(-1, 1, size=length))

def noise(length=10000):
    return np.random.normal(size=length)

def periodic(length=10000, period=100):
    pattern = random_walk(period)
    ind = np.arange(length)
    return pattern[ind % period]
    

In [ ]:
syn_datasets = []
np.random.seed(2)
for i in range(1000):
    length = np.random.randint(100, 100000)
    # hardcoded 5000 as max period length in code
    period = np.random.randint(5, min(2000, length // 10))
    period_strength = np.random.uniform(0, 10)
    noise_strength = np.random.uniform(0, 10)
    sig = period_strength * periodic(period=period, length=length) + random_walk(length=length) + noise_strength * noise(length=length)
    mine, confirmed = find_length(sig, max_filter=True)
    mine_diff = find_length_diff(sig)
    theirs = find_length_rank(sig)
    syn_datasets.append([sig, length, period, noise_strength, period_strength, mine, mine_diff, confirmed, theirs])


In [ ]:
res = pd.DataFrame([x[1:] for x in syn_datasets], columns=["length", "period", "noise_strength", "period_strength", "mine", "mine_diff", "confirmed", "theirs"])

In [ ]:
res

In [ ]:
(res.period == res.mine).mean()

In [ ]:
res_confirmed = res[res.confirmed]

In [ ]:
(res_confirmed.period == res_confirmed.mine).mean()

In [ ]:
(res.period == res.mine_diff).mean()

In [ ]:
from dabl import plot
#plot(X=res, y=res.period==res.mine)

In [ ]:
interesting = res[(res.period_strength > 1) & (res.mine != res.period)]

In [ ]:
interesting = res_confirmed[res_confirmed.period != res_confirmed.mine]

In [ ]:
interesting

In [ ]:
for i in interesting.index[:20]:
    sig, length, period, noise_strength, period_strength, mine, mine_diff, confirmed, theirs = syn_datasets[i]
    plt.figure(figsize=(20, 5))
    plt.title(f"{i} True: {period}  mine: {mine}  theirs: {theirs}")
    plt.plot(sig)

    plt.figure(figsize=(20, 5))
    #n_lags = max(min(5000, 2 * period), 2 * mine)
    #n_lags = max(20 * period, 2 * mine)
    n_lags = 5000
    auto_corr = acf(sig, nlags=n_lags, fft=True)
    plt.plot(auto_corr)
    peaks, _ = find_peaks(auto_corr)
    new_mine = 0
    if len(peaks):
        prominences = peak_prominences(auto_corr, peaks)[0]
        widths = peak_widths(auto_corr, peaks)[0]
        masked_inds = np.where(auto_corr[peaks] >= np.maximum.accumulate(auto_corr[peaks][::-1])[::-1])[0]
        #masked_inds = np.arange(len(peaks))
        #prominence_threshold = np.percentile(prominences, 90)
        sorted_prominences = np.sort(prominences)
        if len(prominences) < 2:
            good_max = sorted_prominences[-1]
        else:
            good_max = sorted_prominences[-2]
        prominence_threshold = good_max - 2 * prominences[masked_inds].std()
        if len(prominences) >= 10:
            prominence_threshold = np.minimum(prominence_threshold, sorted_prominences[-10])      
        pruned_inds = masked_inds[prominences[masked_inds] > prominence_threshold]
        #pruned_peaks = pruned_peaks[widths[pruned_peaks] > widths_threshold]
        
        for p in pruned_inds:
            plt.text(peaks[p], auto_corr[peaks[p]], f"p: {prominences[p]:.2f} w: {widths[p]:.2f}")
        mode = stats.mode(np.diff(np.sort(peaks[pruned_inds])[:20]))
        if mode.count > 1 and mode.mode in peaks[pruned_inds] or mode.count > 3:
            new_mine = mode.mode
        # msg = ""
        # if widths[prominent_peak_idx] > peaks[prominent_peak_idx]:
        #     msg = msg + " early"
        # if prominences[prominent_peak_idx] < 0.1:
        #     msg = msg + " too low"
        # plt.title(msg)
        # plt.text(peaks[prominent_peak_idx], autocorr[peaks[prominent_peak_idx]], f"p: {prominences[prominent_peak_idx]:.2f} w: {widths[prominent_peak_idx]:.2f}")
        plt.plot(peaks[pruned_inds], auto_corr[peaks[pruned_inds]], 'o', c='k', alpha=0.5)
    # plt.hlines([2.576 / np.sqrt(len(sig))], xmin=0, xmax=len(autocorr))
    if mine !=0:
        plt.plot([mine], auto_corr[mine], 'o', c='g')
    if new_mine != 0:
        plt.plot([new_mine], auto_corr[new_mine], 'o', c='y', markersize=10)
    #plt.plot([theirs], autocorr[theirs], 'o', c='r')
    plt.plot([period], auto_corr[period], 'o', c='r')
    #plt.xlim(0, min(max(10 * period, 2 * mine), 5000))
    plt.title(f"mode: {mode}, prominence max: {prominences.max()}")

In [ ]:
prominences

In [ ]:
def find_length_b(data, prominence_percentile=90, n_lags=5000):
    #a, b = np.quantile(data, [0.001, 0.999])
    #data_clipped = np.clip(data, a, b)
    data_clipped = data
    auto_corr = acf(data_clipped, nlags=n_lags, fft=True)
    peaks, _ = find_peaks(auto_corr)

    prominences = peak_prominences(auto_corr, peaks)[0]
    widths = peak_widths(auto_corr, peaks)[0]

    if len(prominences):
        prominent_peak_idx = np.argmax(prominences)
    else:
        return 0

    # easy mode assumption, mostly if there's only one periodicity
    masked_inds = np.where(auto_corr[peaks] >= np.maximum.accumulate(auto_corr[peaks][::-1])[::-1])[0]
    result = peaks[masked_inds[np.argmax(prominences[masked_inds])]]
    if len(prominences) < 2:
        good_max = np.max(prominences)
    else:
        good_max = np.sort(prominences)[-2]
    print(good_max)
    prominence_threshold = good_max - 2 * prominences[masked_inds].std()
    print(prominence_threshold)
    pruned_inds = masked_inds[prominences[masked_inds] > prominence_threshold]

    mode = stats.mode(np.diff(np.sort(peaks[pruned_inds])))
    if mode.count > 1 and mode.mode in peaks[pruned_inds] or mode.count > 3:
        result = mode.mode

    if 4 * result > n_lags and 4 * result < len(data):
        # we didn't see enough lags for robust detection
        result = find_length(data, prominence_percentile=prominence_percentile, n_lags=result * 4)         
    
    return result

In [ ]:
find_length_b(syn_datasets[6][0])

In [ ]:
prominence_threshold

In [ ]:
 np.sort(prominences)[-2]

In [ ]:
np.sort(prominences)[-2]

In [ ]:
np.max(prominences)